In [57]:
from collections import Counter
import numpy as np
from threading import Thread
from multiprocessing import Process, Manager, freeze_support, set_start_method, Array, Queue
from time import time, sleep
import concurrent.futures
from find_min_helper import find_minimum
import multiprocessing, math
import pandas as pd

In [58]:
def find_minimum_sequential(A):
    N = len(A)
    curr_min = A[0]
    for i in range(1,N):
        curr_min = min(curr_min,A[i])
    return curr_min

In [59]:
def find_minimum_for_slice(A, start, end, res):
    curr_min = A[start]
    for i in range(start+1,end):
        curr_min = min(curr_min,A[i])
    res.append(curr_min)


def find_minimum_threads(A,cores):
    N = len(A)
    steps = math.ceil(N//cores)#number of rows should be processed by each thread
    start = 0
    threads = []
    res = []

    if __name__ == '__main__':
        while start<N:
            end = N if start+steps > N else start+steps
            t = Thread(target=find_minimum_for_slice, args = [A,start,end,res])
            threads.append(t)
            start = end #update row number for the start of next process
        for t in threads: t.start()
        for t in threads: t.join()
        return find_minimum_sequential(A)

In [60]:
def find_minimum_multiprocessing(A,cores):
    N = len(A)
    #res = []
    steps = N//cores
    start = 0
    processes = []
    
    if __name__ == '__main__':
        queue = Queue()
        for c in range(cores):
            end = start+steps
            if cores-c==1:end = N
            p = Process(target=find_minimum, args = [A[start:end],queue])
            processes.append(p)
            start = end
        for p in processes: p.start()
        for p in processes: p.join()
        curr_min = float('inf')
        for c in range(cores):curr_min = min(curr_min,queue.get())
        #print(return_dict.values())
        return curr_min

In [63]:
def compare_proc_sizes_for_N(N):
    procs = [1,2,3,4]
    df_compare_add_value = pd.DataFrame(columns=['Process count','Sequential','Multithreading','Multiprocessing with sharing objects'])
    A = np.random.random(size = (N))
    for i in range(len(procs)):
        proc = procs[i]
        t0 = time()
        C = find_minimum_sequential(A)
        t1 = time()
        #print(C)
    
        D = find_minimum_threads(A,proc)
        t2 = time()

        E = find_minimum_multiprocessing(A,proc)
        t3 = time()
        #print(E)

        df_compare_add_value.loc[i] = [proc, t1-t0, t2-t1, t3-t2]
    return df_compare_add_value


In [64]:
df_pow_4 = compare_proc_sizes_for_N(pow(10,4))
df_pow_4

,Process count,Sequential,Multithreading,Multiprocessing with sharing objects
0,1.0,0.006565,0.010062,0.293136
1,2.0,0.004193,0.009064,0.285270
2,3.0,0.004095,0.008974,0.426080
3,4.0,0.003995,0.009122,0.477345


In [65]:
df_pow_5 = compare_proc_sizes_for_N(pow(10,5))
df_pow_5

,Process count,Sequential,Multithreading,Multiprocessing with sharing objects
0,1.0,0.053282,0.084128,0.295096
1,2.0,0.040613,0.084628,0.515870
2,3.0,0.043122,0.084012,0.764834
3,4.0,0.040070,0.083333,0.990363


In [66]:
df_pow_6 = compare_proc_sizes_for_N(pow(10,6))
df_pow_6

,Process count,Sequential,Multithreading,Multiprocessing with sharing objects
0,1.0,0.425124,0.812730,0.756186
1,2.0,0.413452,0.858176,0.771507
2,3.0,0.411789,0.807878,1.078250
3,4.0,0.483531,0.842843,1.292387


In [67]:
df_pow_7 = compare_proc_sizes_for_N(pow(10,7))
df_pow_7

,Process count,Sequential,Multithreading,Multiprocessing with sharing objects
0,1.0,4.238465,8.538984,4.849828
1,2.0,4.207019,8.753615,5.570831
2,3.0,4.858110,8.566062,3.211571
3,4.0,4.373350,8.884650,3.765510


In [68]:
df_pow_8 = compare_proc_sizes_for_N(pow(10,8))
df_pow_8

,Process count,Sequential,Multithreading,Multiprocessing with sharing objects
0,1.0,43.114112,84.515446,42.904703
1,2.0,42.764930,87.352116,26.680857
2,3.0,45.522161,85.921550,27.161172
3,4.0,44.926688,88.505129,31.119421
